In [1]:
import numpy as np 
import pandas as pd
import sys
import xarray as xr 
import matplotlib.pyplot as plt 
import matplotlib.dates as mdates 
import matplotlib as mpl
from datetime import datetime
import pprint
print("Modules imported")

Modules imported


In [2]:
# generate a csv with the paths to all the files I need to pull from the cirrus server
#example link: https://cirrus.ucsd.edu/~pierce/LOCA2_CA_hybrid/LOCA2_CA_hybrid/CESM2-LENS/0p0625deg/r1i1p1f1/historical/psl/psl.CESM2-LENS.historical.r1i1p1f1.1950-2014.LOCA2_CA_hybrid_v20230115.nc

In [3]:
#theoretically the only thing you need to change
var='wspeed' #DTR,pr,psl,rsds,tasmax,tasmin,uas,vas, wspeed

In [4]:
#iterate through the models, SSPs, and intializations
models=['ACCESS-CM2','CNRM-ESM2-1','EC-Earth3','EC-Earth3-Veg','GFDL-ESM4','HadGEM3-GC31-LL','INM-CM5-0','IPSL-CM6A-LR','MIROC6','MPI-ESM1-2-HR','MRI-ESM2-0']
ssps=['historical','ssp245','ssp370','ssp585']
init=['r1i1p1f1','r2i1p1f1','r3i1p1f1','r4i1p1f1','r5i1p1f1','r6i1p1f1','r7i1p1f1','r8i1p1f1','r9i1p1f1','r10i1p1f1','r1i1p1f2','r1i1p1f3','r2i1p1f3','r3i1p1f3']

In [5]:
index=list(np.arange(0,len(init)))
result = [str(x) + str(y) for x, y in zip(index, init)]
print(result)

['0r1i1p1f1', '1r2i1p1f1', '2r3i1p1f1', '3r4i1p1f1', '4r5i1p1f1', '5r6i1p1f1', '6r7i1p1f1', '7r8i1p1f1', '8r9i1p1f1', '9r10i1p1f1', '10r1i1p1f2', '11r1i1p1f3', '12r2i1p1f3', '13r3i1p1f3']


In [6]:
#then need a multiplication array to indicate which models and SSPs include which initializations (4 per model), should this be a 3D array?
n1=len(models)
n2=len(ssps)
n3=len(init)
windmask=[[[0 for k in range(n3)] for j in range(n2)] for i in range(n1)]

In [7]:
#pprint.pprint(windmask)

In [8]:
#now assign where there are ones
#is the index order model, ssp, init
windmask=np.array(windmask)
#go by model to match sheet
windmask[0,:,0]=1

windmask[1,:,10]=1

windmask[2,0,0:4]=1
windmask[2,1,[0,1,3]]=1
windmask[2,2,[0,3]]=1
windmask[2,3,[0,2,3]]=1

windmask[3,0,0:5]=1
windmask[3,1,0:5]=1
windmask[3,2,0:4]=1
windmask[3,3,0:4]=1

windmask[4,0,0]=1
windmask[4,1,0]=1
windmask[4,2,0]=1
windmask[4,3,0]=1

windmask[5,0,11:]=1
windmask[5,1,11]=1
#no 370 SSPt for HadGEM3-GC31-LL
windmask[5,3,11:]=1

windmask[6,0,0:5]=1
windmask[6,1,0]=1
windmask[6,2,0:5]=1
windmask[6,3,0]=1

windmask[7,0,0:10]=1
windmask[7,1,0:5]=1
windmask[7,2,0:10]=1
windmask[7,3,0:4]=1

windmask[8,0,0:5]=1
windmask[8,1,0:3]=1
windmask[8,2,0:3]=1
windmask[8,3,0:5]=1

windmask[9,0,[0,1,2,3,4,9]]=1
windmask[9,1,0:2]=1
windmask[9,2,[0,1,2,3,4,9]]=1
windmask[9,3,0:2]=1

windmask[10,0,0:5]=1
windmask[10,1,0]=1
windmask[10,2,0:5]=1
windmask[10,3,0]=1

pprint.pprint(windmask)

array([[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],

       [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]],

       [[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],

       [[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],

       [[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    

In [9]:
root ='https://cirrus.ucsd.edu/~pierce/LOCA2_CA_hybrid/LOCA2_CA_hybrid/'

In [10]:
# generate a csv with the paths to all the files I need to pull from the cirrus server
#example link: https://cirrus.ucsd.edu/~pierce/LOCA2_CA_hybrid/LOCA2_CA_hybrid/CESM2-LENS/0p0625deg/r1i1p1f1/historical/psl/psl.CESM2-LENS.historical.r1i1p1f1.1950-2014.LOCA2_CA_hybrid_v20230115.nc

In [11]:
# # assign list
# l = ['Geeks','for','Geeks!']

# # open file
# with open('gfg.txt', 'w+') as f:
    
#     # write elements of list
#     for items in l:
#         f.write('%s\n' %items)
    
#     print("File written successfully")


# # close the file
# f.close()    


In [12]:
#actually if I want to use the regex notation I need to path the website root separately from the file path so maybe need to write those to separate csvs
#iterate through to make the file name and then path
mn=0
sn=0
itn=0

sitefilename='site_'+var+'.txt'
filenamefilename='filename_'+var+'.txt'

site=open(sitefilename,'w+')
filename=open(filenamefilename,'w+')

for m in models:
    sn=0
    for s in ssps:
        itn=0
        for it in init:
            
            if windmask[mn,sn,itn]==1:
                fname=var+'.'+m+'.'+s+'.'+it+'*'+'v20230115.nc' #see if the star works bc there are a few files with different year ranges for different models
                path=root+m+'/0p0625deg/'+it+'/'+s+'/'+var+'/'
                # site.write('%s\n' %path)
                # filename.write('%s\n' %fname)
                # print('%s\n' %path)
                # print('"%s"\n' %fname)
                site.write('%s\n' %path)
                filename.write('"%s"\n' %fname)
                
            itn=itn+1
        sn=sn+1
    mn=mn+1

site.close()
filename.close()

In [13]:
#iterate through to make the file name and then path
mn=0
sn=0
itn=0
for m in models:
    sn=0
    for s in ssps:
        itn=0
        for it in init:
            
            if windmask[mn,sn,itn]==1:
                fname=var+'.'+m+'.'+s+'.'+it+'*'+'v20230115.nc' #see if the star works bc there are a few files with different year ranges for different models
                path=root+m+'/0p0625deg/'+it+'/'+s+'/'+var+'/'+fname
                #print(path)
                
            itn=itn+1
        sn=sn+1
    mn=mn+1
        

In [14]:
#test on the download node
#https://cirrus.ucsd.edu/~pierce/LOCA2_CA_hybrid/LOCA2_CA_hybrid/ACCESS-CM2/0p0625deg/r1i1p1f1/ssp245/psl/psl.ACCESS-CM2.ssp245.r1i1p1f1*v20230115.nc